# Verity AI SDK Examples

This notebook demonstrates how to use the Verity AI Python SDK to interact with various AI services including chat completions, model management, SQL queries, file management, and document search.

## Prerequisites
1. Install the Verity AI SDK
2. Set up your API key
3. Ensure you have access to the required knowledge bases and databases


In [3]:
import requests
import tempfile
import json
from pathlib import Path

In [ ]:
# Install the Verity AI client
%pip install --upgrade verity-ai-client


In [5]:
import os
import verity_ai_pyc
from verity_ai_pyc.rest import ApiException
import getpass

# Set your API key (you can also set this as an environment variable)
if not os.getenv("API_KEY"):
    api_key = getpass.getpass("Enter your Verity AI API key: ")
    os.environ["API_KEY"] = api_key
else:
    print("✅ API key found in environment variables")

# Configure the client
configuration = verity_ai_pyc.Configuration(
    host="https://chat.veritylabs.ai"
)
configuration.api_key['XAPIKeyAuth'] = os.environ["API_KEY"]

print("🔧 Configuration complete!")


✅ API key found in environment variables
🔧 Configuration complete!


## 1. Models API - List Available Models

 Let's start by exploring what models are available:

In [6]:

def list_available_models():
    """List all available models."""
    with verity_ai_pyc.ApiClient(configuration) as api_client:
        models_api = verity_ai_pyc.ModelsApi(api_client)
        models = models_api.list_models_rag_generation_models_get()
        print(f"Available models: {models}")
        return models

# Run the example
available_models = list_available_models()


Available models: ['anthropic_claude_3_haiku_v1', 'anthropic_claude_3_5_sonnet_v1', 'anthropic_claude_3_7_sonnet_v1']


## 2. File Upload and Delete Examples

These examples show how to upload and delete files using the Verity AI API:


### Unstructured Data 

In [7]:
def simple_upload_unstructured_example(file_path):
    """Simple example of uploading an unstructured file."""

    # Configure the client
    configuration = verity_ai_pyc.Configuration(
        host="https://chat.veritylabs.ai"
    )
    configuration.api_key['XAPIKeyAuth'] = os.environ["API_KEY"]

    # Create API client
    with verity_ai_pyc.ApiClient(configuration) as api_client:
        file_management_api = verity_ai_pyc.FileManagementApi(api_client)
        
        try:
            response = file_management_api.upload_file_fileman_data_upload_post(
                file=file_path,
                storage_type="unstructured",
                knowledge_base="test_knowledge_base"
            )
            
            print(f"✅ Successfully uploaded {file_path}")
            print(f"Response: {response}")
            return response
        except ApiException as e:
            print(f"❌ Error uploading file: {e}")
            return None

file_path = "../../../data/demo_datasets/test/llava.pdf"
simple_upload_unstructured_example(file_path)

✅ Successfully uploaded ../../../data/demo_datasets/test/llava.pdf
Response: message='File uploaded successfully' s3_path='s3://120569609191-growth-unstructured/veritydemo/test_knowledge_base/llava.pdf'


UploadFileResponse(message='File uploaded successfully', s3_path='s3://120569609191-growth-unstructured/veritydemo/test_knowledge_base/llava.pdf')

In [8]:
def list_files(storage_type="unstructured", page=1, page_size=10):
    """List files in the knowledge base."""
    with verity_ai_pyc.ApiClient(configuration) as api_client:
        file_management_api = verity_ai_pyc.FileManagementApi(api_client)
        response = file_management_api.list_files_get_fileman_data_list_get(
            storage_type=storage_type, page=page, page_size=page_size
        )
        
        print(f"Found {len(response.files)} files:")
        for i, file_info in enumerate(response.files, 1):
            filename = getattr(file_info, 'filename', 'Unknown')
            size_mb = getattr(file_info, 'size_mb', 0)
            print(f"{i}. {filename} ({size_mb:.2f} MB)")
        return response

# List files
file_list = list_files(page_size=5)

Found 3 files:
1. Contracts/SampleContract-Shuttle.pdf (0.05 MB)
2. Test KB/TED-Platform-for-AI-Search-Seamless-Data-Integration-Semantic-Search-and-AI-Powered-Insights-for-the.pdf (7.89 MB)
3. Veirty_docs/TED-Platform-for-AI-Search-Seamless-Data-Integration-Semantic-Search-and-AI-Powered-Insights-for-the.pdf (7.89 MB)


In [9]:
def simple_delete_unstructured_example(filenames):
    """Simple example of deleting an unstructured file."""
    print("\n🗑️ Simple Delete Unstructured File Example")
    print("-" * 40)
    
    # Configure the client
    configuration = verity_ai_pyc.Configuration(
        host="https://chat.veritylabs.ai"
    )
    configuration.api_key['XAPIKeyAuth'] = os.environ["API_KEY"]

    # Create API client
    with verity_ai_pyc.ApiClient(configuration) as api_client:
        file_management_api = verity_ai_pyc.FileManagementApi(api_client)
        
        try:
            # Create delete request
            delete_request = verity_ai_pyc.DeleteRequest(
                filenames=filenames
            )
            
            response = file_management_api.delete_files_fileman_data_delete_delete(
                storage_type="unstructured",
                delete_request=delete_request,
                base_path="test_knowledge_base"
            )
            
            print("✅ Successfully deleted unstructured file")
            print(f"Response: {response}")
            return response
        except ApiException as e:
            print(f"❌ Error deleting unstructured file: {e}")
            return None


filenames = ["llava.pdf"]
simple_delete_unstructured_example(filenames)


🗑️ Simple Delete Unstructured File Example
----------------------------------------
✅ Successfully deleted unstructured file
Response: status='success' deleted_files=['llava.pdf']


DeleteResponse(status='success', deleted_files=['llava.pdf'])

### Structured Data

In [10]:
def simple_upload_structured_example(filepath):
    """Simple example of uploading a structured file."""
    
    # Configure the client
    configuration = verity_ai_pyc.Configuration(
        host="https://chat.veritylabs.ai"
    )
    configuration.api_key['XAPIKeyAuth'] = os.environ["API_KEY"]

    # Create API client
    with verity_ai_pyc.ApiClient(configuration) as api_client:
        file_management_api = verity_ai_pyc.FileManagementApi(api_client)
        
        try:            
            response = file_management_api.upload_file_fileman_data_upload_post(
                file=filepath,
                storage_type="structured",
                database_name="test_db",
                table_name="test_table"
            )
            
            print(f"✅ Successfully uploaded {filepath}")
            print(f"Response: {response}")
            return response
        except ApiException as e:
            print(f"❌ Error uploading structured file: {e}")
            return None
        
filepath = "../../../data/structured/pharma/MIMIC_CALLOUT/MIMIC_CALLOUT.csv"
simple_upload_structured_example(filepath)

✅ Successfully uploaded ../../../data/structured/pharma/MIMIC_CALLOUT/MIMIC_CALLOUT.csv
Response: message='File uploaded successfully' s3_path='s3://120569609191-growth-structured/veritydemo/test_db/test_table/MIMIC_CALLOUT.csv'


UploadFileResponse(message='File uploaded successfully', s3_path='s3://120569609191-growth-structured/veritydemo/test_db/test_table/MIMIC_CALLOUT.csv')

In [11]:
def simple_delete_structured_example(filenames):
    """Simple example of deleting a structured file."""

    # Configure the client
    configuration = verity_ai_pyc.Configuration(
        host="https://chat.veritylabs.ai"
    )
    configuration.api_key['XAPIKeyAuth'] = os.environ["API_KEY"]

    # Create API client
    with verity_ai_pyc.ApiClient(configuration) as api_client:
        file_management_api = verity_ai_pyc.FileManagementApi(api_client)
        
        try:
            # Create delete request
            delete_request = verity_ai_pyc.DeleteRequest(
                filenames=filenames
            )
            
            response = file_management_api.delete_files_fileman_data_delete_delete(
                storage_type="structured",
                delete_request=delete_request,
                base_path="test_db",
                sub_path="test_table"
            )
            
            print("✅ Successfully deleted structured file")
            print(f"Response: {response}")
            return response
        except ApiException as e:
            print(f"❌ Error deleting structured file: {e}")
            return None
filenames = ["MIMIC_CALLOUT.csv"]
simple_delete_structured_example(filenames)

✅ Successfully deleted structured file
Response: status='success' deleted_files=['MIMIC_CALLOUT.csv']


DeleteResponse(status='success', deleted_files=['MIMIC_CALLOUT.csv'])

## 3. Chat Completions API - Unstructured Data

This example shows how to use the chat completion API with unstructured data (general knowledge):


In [13]:
def chat_with_unstructured_data(question="What is artificial intelligence?"):
    """Chat completion with unstructured data."""
    with verity_ai_pyc.ApiClient(configuration) as api_client:
        completions_api = verity_ai_pyc.CompletionsApi(api_client)
        chat_request = verity_ai_pyc.ChatCompletionRequestPublic(
            model="anthropic_claude_3_5_sonnet_v1",
            messages=[verity_ai_pyc.Message(role="user", content=question)],
            data_type="unstructured",
            knowledge_base=verity_ai_pyc.KnowledgeBase("all")
        )
        response = completions_api.create_chat_completion_rag_generation_chat_completions_post(chat_request)
        print(f"Question: {question}")
        print(f"Answer: {response.messages[0].content}")
        return response

# Run the example
chat_response = chat_with_unstructured_data("What is Verity Labs?")


Question: What is Verity Labs?
Answer: Based on the provided context, Verity Labs appears to be a company that offers AI-powered solutions for enterprise data management and analysis. While there isn't a direct definition of Verity Labs in the given information, we can infer the following:

1. Verity Labs provides an AI platform that helps businesses connect to their data and automate work processes.

2. They offer solutions for various industries, including:
   - Pharma & Healthcare
   - Financial Services
   - Manufacturing

3. Their functional solutions cover areas such as:
   - Data & Analytics
   - Contact Center Operations
   - General Operations

4. Verity Labs has offices in the United States (Santa Clara, CA) and India (Bengaluru, Karnataka).

5. Their tagline is "Transforming enterprise data into actionable intelligence with AI-powered solutions."

6. They provide a platform with features like the TED Stack (their core technology stack), security features, and flexible deploy

## 4. SQL Chat API - Structured Data

This example shows how to query structured data (databases) using natural language:


In [14]:
def chat_with_sql_data(question="How many records are in the database?"):
    """Chat completion with SQL/structured data."""
    with verity_ai_pyc.ApiClient(configuration) as api_client:
        completions_api = verity_ai_pyc.CompletionsApi(api_client)
        chat_request = verity_ai_pyc.ChatCompletionRequestPublic(
            model="anthropic_claude_3_5_sonnet_v1",
            messages=[verity_ai_pyc.Message(role="user", content=question)],
            data_type="structured",
            database_name="veritydemo_mimic"
        )
        response = completions_api.create_chat_completion_rag_generation_chat_completions_post(chat_request)
        print(f"Question: {question}")
        print(f"Answer: {response.messages[0].content}")
        return response

# Run the example
sql_response = chat_with_sql_data()


Question: How many records are in the database?
Answer: # Query Results Summary

## Overview
- **Total Records**: 77,815

## Key Insights

This query result provides a single, important metric:

- The database or table being queried contains **77,815 total records**.

## Context and Implications

1. **Data Volume**: This is a substantial dataset, with nearly 80,000 records. This volume suggests:
   - A comprehensive collection of information
   - Potential for rich data analysis and insights
   - Likely covers a significant time period or wide range of entries

2. **Data Management**: With this many records, considerations should be given to:
   - Efficient data storage and retrieval methods
   - Potential need for data indexing or optimization for query performance
   - Regular maintenance and data quality checks

3. **Analysis Opportunities**: This large dataset offers:
   - Potential for identifying trends or patterns over time
   - Possibilities for segmentation or categorization o

## 5. Document Search API - Retrieve Relevant Documents

This example shows how to search for relevant documents in your knowledge base:


In [20]:
def search_documents(query="apple", top_k=5):
    """Search for documents in the knowledge base."""
    # Create API client
    with verity_ai_pyc.ApiClient(configuration) as api_client:
        unstructured_api = verity_ai_pyc.UnstructuredApi(api_client)
        
        try:
            # Create retrieval request  
            request = verity_ai_pyc.RetrievalRequestPublic(
                query="apple",
                knowledge_base=verity_ai_pyc.KnowledgeBase1("all"),  # Use KnowledgeBase1 for RetrievalRequestPublic
                top_k=5
            )
            
            response = unstructured_api.retrieve(request)
            print("✅ Document search completed successfully!")
            print(response)
            return response
        except ApiException as e:
            print(f"❌ Error: {e}")
            return None


# Search documents
search_results = search_documents()

✅ Document search completed successfully!
{'status': 'success', 'results': [{'_id': 'MtaT3ZYBSJd5sHbkAcWd', 'paragraph': 'Accounts Receivable\n\nTrade Receivables\n\nThe Company has considerable trade receivables outstanding with its third-party cellular network carriers, wholesalers, retailers, resellers, small and mid-sized\nbusinesses  and  education,  enterprise  and  government  customers.  The  Company  generally  does  not  require  collateral  from  its  customers;  however,  the\nCompany will require collateral or third-party credit support in certain instances to limit credit risk. In addition, when possible, the Company attempts to limit credit\nrisk on trade receivables with credit insurance for certain customers or by requiring third-party financing, loans or leases to support credit exposure. These credit-\nfinancing arrangements are directly between the third-party financing company and the end customer. As such, the Company generally does not assume any\nrecourse or cre

## 6. Agent Chat Completion - Advanced AI Interactions

This example shows how to use the agent chat completion API for more advanced AI interactions:


In [21]:
def simple_agent_chat_example(agent_id):
    """Simple example of agent chat using agent_flag=True and agent_id."""

    with verity_ai_pyc.ApiClient(configuration) as api_client:
        completions_api = verity_ai_pyc.CompletionsApi(api_client)
        

        chat_request = verity_ai_pyc.ChatCompletionRequestPublic(
            model="anthropic_claude_3_5_sonnet_v1",
            messages=[verity_ai_pyc.Message(role="user", content="Hello! What can you help me with?")],
            agent_flag=True,  # Enable agent mode
            agent_id=agent_id,  # Specify which agent to use
            data_type="unstructured",
            knowledge_base=verity_ai_pyc.KnowledgeBase("all")  # Pass as KnowledgeBase object
        )

        try:
            response = completions_api.create_chat_completion_rag_generation_chat_completions_post(chat_request)
            print("✅ Agent Response:", response.messages[0].content)
            return response
        except ApiException as e:
            print(f"❌ Error: {e}")
            print("💡 Make sure to replace 'CUS_your_agent_id_here' with a real agent ID")
            return None

In [22]:
# Example with a specific agent ID (replace with your actual agent ID) from the portal
agent_id = "CUS_d551e1ae-3792-423f-950c-bc3a2ef5680e"  
simple_agent_chat_example(agent_id)

✅ Agent Response: Hello! I'm an AI assistant created by Anthropic to help with a variety of tasks. I can assist you with things like:

- Information lookup and research
- Writing and editing help (e.g. documents, emails, articles)
- Analysis and problem-solving
- Creative ideation and brainstorming
- And much more!

Please feel free to ask me about any topic or task you need help with, and I'll do my best to provide useful information or guidance. How can I assist you today?


ChatCompletionResponse(id='a38ec6db-c290-463c-88fb-c62381f632fb', created=1749720699, model='anthropic_claude_3_5_sonnet_v1', messages=[ChatCompletionMessage(role='assistant', content="Hello! I'm an AI assistant created by Anthropic to help with a variety of tasks. I can assist you with things like:\n\n- Information lookup and research\n- Writing and editing help (e.g. documents, emails, articles)\n- Analysis and problem-solving\n- Creative ideation and brainstorming\n- And much more!\n\nPlease feel free to ask me about any topic or task you need help with, and I'll do my best to provide useful information or guidance. How can I assist you today?", metadata={'stage': 'final_answer'})], structured_data=[], sources=[], usage=Usage(prompt_tokens=7, completion_tokens=81, total_tokens=88), metadata={'history': [{'step': 1, 'timestamp': '2025-06-12T09:31:38.119697Z', 'role': 'user', 'content': 'Hello! What can you help me with?', 'category': None, 'metadata': {}}, {'step': 2, 'timestamp': '2